In [2]:
import numpy as np
from numpy import log, exp, sqrt
from scipy.stats import gamma as Gamma_Distribution
from scipy.special import psi
from scipy.special import gamma as Gamma_Function

## Exercise 4.1

In [3]:
# simulate some data
n, k = 500, 3
beta = np.arange(k) + 0.5
X = np.random.rand(n, k)
mu = X.dot(beta)
p = np.random.rand(n)
y = - mu * np.log(1 - p)
# plt.figure()
# plt.hist(y,n/20)
# plt.show()

In [4]:
def logL(beta, X, y):
    u = X.dot(beta)
    l = - (y/u) - log(u)
    return l.sum()

In [5]:
def myqnewton(f, x0, B, searchmeth = 3,stepmeth = "bt" ,maxit = 10000, maxstep = 10000,tol = 1/100000,\
              eps = np.spacing(1),eps0 =1.0, eps1 = 1.e-12, all_x = False):
    '''
    maxit, maxstep, tol,eps0, eps1  = 10000, 10000, 1/10000,1.0,1.e-12
    f: object function and jacobian
    B:  inversed Hessian matrix  
    x0: initial value
    all_x: if we collect x value for plotting
    '''
    x = x0
    if all_x:
        x_list = [x0]
        
    
    A = f(x)
    _is_there_jacobian = (type(A) is tuple) and (len(A) == 2)

    if _is_there_jacobian:
        print('Jacobian was provided by user!')
        fx0,g0 = f(x)
    else:    
        print('Jacobian was not provided by user!')
        fx0 = f(x)
        try:
            g0 = jacobian(f,x)
        except NameError:
            print("jacobian function Not in scope!\n Using identity matrix as jacobian matrix")
            g0 = np.identity(k)
        else:
            print("jacobian function In scope!")    
        
    if np.all(np.abs(g0) < eps): # similar to np.all(g0<eps)
        print("abs(g0)< eps...")
        return x
    
    print("Solving nonlinear equations by using {} search method and {} step method".format(search_methods[searchmeth-1].capitalize(), stepmeth)) 

    print("Start iteration......")
   

    for it in range(maxit):


        d = -np.dot(B, g0)  # search direction, initial d
        
        # https://github.com/randall-romero/CompEcon-python/blob/master/compecon/optimize.py
        if (np.inner(d, g0) / (np.inner(d, d))) < eps1:  # must go uphill
            B =  -np.identity(k) / np.maximum(abs(fx0), 1) # otherwise use
            d = g0 / np.maximum(np.abs(fx0), 1)  # steepest ascent
        # optimize search step length
        s, fx = optstep(stepmeth ,f, x, fx0, g0, d, maxstep)

        if fx <= fx0:

            warnings.warn('Iterations stuck in qnewton')
            #return x
            # reset Hessian and d.
            B =  -np.identity(k) / np.maximum(abs(fx0), 1) # otherwise use
            d = g0.T / np.maximum(abs(fx0), 1)  # steepest ascent
            s, fx = optstep("bt" ,f, x, fx0, g0, d, maxstep)
            if errcode:
                warnings.warn('Cannot find suitable step in qnewton')
                # return x
                # reset to 1 and fx0
                s, fx =  1, fx0

        # update d and x
        d *= s
        x = x + d
        # keep record of x sequence in list
        if all_x:
            x_list.append(x.copy())

        if np.any(np.isnan(x) | np.isinf(x)):
            raise ValueError('NaNs or Infs encountered')

        #  update fx and g again
        if _is_there_jacobian:
            #print('Jacobian was provided by user!')
            fx,g = f(x)
        else:    
            print('Jacobian was not provided by user!')
            fx = f(x)
            try:
                g = jacobian(f,x)
            except NameError:
                print("jacobian function Not in scope!\n Using identity matrix as jacobian matrix")
                g = np.identity(k)
            else:
                print("jacobian function In scope!")


        # Test convergence using Marquardt's criteria and gradient test
        if ((fx - fx0) / (abs(fx) + eps0) < tol and
                np.all(np.abs(d) / (np.abs(x) + eps0) < tol)) or\
                np.all(np.abs(g) < eps):
                print("Meet the tol. x: ", x)
                #break
                if all_x:
                    return x, x_list
                else:
                    return x



        # Update inverse Hessian
        u = g - g0  # change in Jacobian
        ud = np.inner(u, d)

        # pick a search method
        #print("Please specify one search method: 1:steepest ascen;2: DFP;3:BFGS")
        if np.all(np.abs(ud) < eps):
            B =  -np.identity(k) / np.maximum(abs(fx0), 1) # otherwise use
        else:
            if searchmeth == 1 and np.abs(ud) < eps: # steepest ascent
                B =  -np.identity(k) / np.maximum(abs(fx), 1)
            elif searchmeth == 2: # DFP
                v = B.dot(u)
                B += np.outer(d, d) / ud - np.outer(v, v) / np.inner(u, v) 
            elif searchmeth == 3: # BFGS
                w = d - B.dot(u)
                wd = np.outer(w, d)
                B += ((wd + wd.T) - (np.inner(u, w) * np.outer(d, d)) / ud) / ud


        # Update iteration
        fx0 = fx
        g0 = g
        print("finish {}th iteration...".format(it))   

    # end of iteration if exceed the maxit
    if it >= maxit:
        warnings.warn('Maximum iterations exceeded in qnewton')
        return x

In [ ]:
L = OP(logL, np.ones(k),X, y)
beta_hat = myqnewton(logL, np.ones(k),X, y)
print('Looking for the maximum likelihood:    beta = ', beta_hat)

In [ ]:
def dlogL(beta, X, y):
    u = X.dot(beta)
    temp = ((y - u) / u ** 2)
    dl = temp[:, np.newaxis] * X
    return dl.sum(0)

## Exercise 4.2

In [ ]:

# simulate some data
n = 500
a = 5.0
b = 2.0
x_data = Gamma_Distribution.rvs(a, scale=1/b, size=n)
Y1 = x_data.mean()
Y2 = exp(log(x_data).mean())
b_hat = lambda a0: a0 / Y1
def dlogL(theta):
    return log(theta) - log(Y1 / Y2) - psi(theta)
a0 = 1.1 # initial guess

estimator = NLP(dlogL, a0, print=True, all_x=True)



# estimator = MCP(dlogL, 0, np.inf, a0, print=True, all_x=True)
a_hat = estimator.zero()
print(estimator.x_sequence)
print(b_hat(estimator.x_sequence))
y1y2 = np.linspace(1.1, 3, 48)
dlogL2 = lambda theta, y12: log(theta) - log(y12) - psi(theta)
ttheta = np.array([NLP(dlogL2, a0, k).zero() for k in y1y2])
plt.figure()
plt.plot(y1y2, ttheta)
plt.xlabel('Y1 / Y2')
plt.ylabel('theta1')
plt.show()
# Solve it using the MLE object
def logL(theta, x):
    n = x.size
    a, b = theta
    return n*a*log(b) + (a-1)*log(x).sum() - b*x.sum() - n*log(Gamma_Function(a))
mle = MLE(logL, np.ones(2), x_data)
mle.estimate()
print('theta1 = {:.4f}, theta1 = {:.4f}'.format(*mle.beta))
print('Estimated Covariance = \n', mle.Sigma)
print('Confidence intervals\n', mle.ci())

## Exercise 4.3

In [ ]:



treasury_tau = np.array([0.25, 0.5, 1, 2, 3, 5, 7, 10, 30])

treasury_r = np.array(
    [[4.44, 4.49, 4.51, 4.63, 4.63, 4.62, 4.82, 4.77, 5.23],
     [4.45, 4.48, 4.49, 4.61, 4.61, 4.60, 4.84, 4.74, 5.16],
     [4.37, 4.49, 4.53, 4.66, 4.66, 4.65, 4.86, 4.76, 5.18],
     [4.47, 4.47, 4.51, 4.57, 4.57, 4.57, 4.74, 4.68, 5.14]])


def Z(r, t, k, a, s):
    gamma = sqrt(k **2 + 2 * s ** 2)
    egt = exp(gamma * t) - 1

    numA = 2 * gamma * exp((gamma + k) * t / 2)
    numB = 2*egt
    den = (gamma + k) * egt + 2 * gamma
    expA = 2 * k * a / (s ** 2)
    A = (numA / den) ** expA
    B = numB / den
    Z = A * exp(-B * r)
    return Z

def ss(x, r, tau):
    k, a, s = x
    resid = r + 100 * log(Z(r / 100, tau, k, a, s)) / tau
    return -(resid ** 2).sum()


def ss2(x, r, tau):
    tmp = lambda x: ss(x, r, tau)
    return jacobian(tmp, x)[0]





x0 = np.array([0.51, 0.05, 0.12])

hola = MCP(ss2, np.zeros(3), np.ones(3), x0, treasury_r[0], treasury_tau)
x = hola.zero(print=True)
print(x)

objective = OP(ss, x0, treasury_r[0], treasury_tau)
objective.qnewton(print=True, all_x=True)
print(objective.x)
print(objective.fnorm)